<a href="https://colab.research.google.com/github/AyushiM1102/Electra_classification_fake_vs_real_news/blob/main/electra_classification_metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Reference tutorial:
# https://velog.io/@na2na8/ELECTRA%EB%A1%9C-Binary-Classification#electra-with-pytorch-lightning

In [2]:
!pip install transformers --quiet

In [3]:
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning --quiet
import pytorch_lightning as pl
print(pl.__version__)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
1.7.0dev


In [4]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!wandb login

wandb: Currently logged in as: amcheyre (ninja-women). Use `wandb login --relogin` to force relogin


In [6]:
import os
import re

import numpy as np
import pandas as pd

import torch
import torchmetrics
import torch.nn as nn
import wandb
from torch.utils.data import DataLoader, Dataset

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import loggers as pl_loggers
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import transformers
from transformers import ElectraForSequenceClassification, ElectraTokenizer, AdamW

device = torch.device("cuda")


In [7]:
!git clone https://github.com/AyushiM1102/Electra_classification_fake_vs_real_news.git

fatal: destination path 'Electra_classification_fake_vs_real_news' already exists and is not an empty directory.


In [8]:
!unzip /content/Electra_classification_fake_vs_real_news/data/WELFake_Dataset.csv.zip -d /content/Electra_classification_fake_vs_real_news/dataset

Archive:  /content/Electra_classification_fake_vs_real_news/data/WELFake_Dataset.csv.zip
replace /content/Electra_classification_fake_vs_real_news/dataset/WELFake_Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/Electra_classification_fake_vs_real_news/dataset/__MACOSX/._WELFake_Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [9]:
# Overview of database
# df = pd.read_csv('/content/Electra_classification_fake_vs_real_news/sample_dataset/train.csv', sep=',')
datapath = f'/content/Electra_classification_fake_vs_real_news/dataset/WELFake_Dataset.csv'
df = pd.read_csv(datapath, sep=',')
df = df.dropna(axis=0)
df.drop_duplicates(inplace=True)

In [10]:
df['label'].value_counts()

1    36509
0    35028
Name: label, dtype: int64

In [11]:
train_size = int(0.7 * len(df))
val_size = int((2/3)*(len(df) - train_size))
test_size = int((1/3)*(len(df) - train_size))
train_size, val_size, test_size

(50075, 14308, 7154)

In [12]:
train_size + val_size + test_size

71537

In [13]:
dataset = shuffle(df)

In [14]:
train_dataset, general_test_dataset  = train_test_split(dataset, train_size=int(0.7 * len(dataset)), test_size=int(0.3 * len(dataset)))

In [15]:
val_dataset, test_dataset = train_test_split(general_test_dataset, train_size=int((2/3) * len(general_test_dataset)), test_size=int((1/3) * len(general_test_dataset)))

In [16]:
len(train_dataset),len(val_dataset),len(test_dataset)

(50075, 14307, 7153)

In [17]:
train_dataset.to_csv('/content/Electra_classification_fake_vs_real_news/dataset/train.csv', index = False)
val_dataset.to_csv('/content/Electra_classification_fake_vs_real_news/dataset/val.csv', index = False)
test_dataset.to_csv('/content/Electra_classification_fake_vs_real_news/dataset/test.csv', index = False)

In [18]:
class ElectraClassificationDataset(Dataset) :
    def __init__(self, path, sep, doc_col, label_col, max_length, num_workers=1, labels_dict=None) :

        self.tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")
        self.max_length = max_length
        self.doc_col = doc_col
        self.label_col = label_col

        # labels, ex : {True : 1, False : 0}
        self.labels_dict = labels_dict

        # dataset
        df = pd.read_csv(path, sep=sep)
        df = df.dropna(axis=0)
        df.drop_duplicates(subset=[self.doc_col], inplace=True)
        self.dataset = df

    def __len__(self) :
        return len(self.dataset)
    
    # Clean text
    def cleanse(self, text) :
        url_pattern = re.compile(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
        processed = url_pattern.sub(' ', text)
        processed = processed.replace('#', '')
        processed = processed.replace('@', '')
        processed = processed.strip()
        return processed

    def __getitem__(self, idx) :
        document = self.cleanse(self.dataset[self.doc_col].iloc[idx])
        #print(document)
        inputs = self.tokenizer(
            document,
            return_tensors='pt',
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            add_special_tokens=True
        )

        if self.labels_dict :
            label = self.labels_dict[self.dataset[self.label_col].iloc[idx]]
        else :
            label = self.dataset[self.label_col].iloc[idx]

        return {
            'input_ids' : inputs['input_ids'][0],
            'attention_mask' : inputs['attention_mask'][0],
            'label' : int(label)
        }

In [19]:
class ElectraClassificationDataModule(pl.LightningDataModule) :
    def __init__(self, train_path, valid_path, test_path, max_length, batch_size, sep,
                doc_col, label_col, num_workers=1, labels_dict=None) :
        super().__init__()
        self.batch_size = batch_size
        self.train_path = train_path
        self.valid_path = valid_path
        self.test_path = test_path
        self.max_length = max_length
        self.doc_col = doc_col
        self.label_col = label_col
        self.sep = sep
        self.num_workers = num_workers
        self.labels_dict = labels_dict

    def setup(self, stage=None) :
      
        self.set_train = ElectraClassificationDataset(self.train_path, sep=self.sep,
                                            doc_col=self.doc_col, label_col=self.label_col,
                                            max_length = self.max_length, labels_dict=self.labels_dict)
        
        self.set_valid = ElectraClassificationDataset(self.valid_path, sep=self.sep,
                                            doc_col=self.doc_col, label_col=self.label_col,
                                            max_length = self.max_length, labels_dict=self.labels_dict)
        
        self.set_test = ElectraClassificationDataset(self.test_path, sep=self.sep,
                                            doc_col=self.doc_col, label_col=self.label_col,
                                            max_length = self.max_length, labels_dict=self.labels_dict)
        

    def train_dataloader(self) :
        train = DataLoader(self.set_train, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=True)
        return train
    
    def val_dataloader(self) :
        val = DataLoader(self.set_valid, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=False)
        return val
    
    def test_dataloader(self) :
        test = DataLoader(self.set_test, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=False)
        return test

In [20]:
# https://medium.com/huggingface/multi-label-text-classification-using-bert-the-mighty-transformer-69714fa3fb3d
# https://huggingface.co/docs/transformers/v4.15.0/en/model_doc/electra#transformers.ElectraForSequenceClassification

In [21]:
class ElectraClassification(pl.LightningModule) :
    def __init__(self, learning_rate) :
        super().__init__()
        self.learning_rate = learning_rate
        self.save_hyperparameters()
        self.electra = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator")

        self.metric_acc = torchmetrics.Accuracy()
        self.metric_f1 = torchmetrics.F1Score(num_classes=2)
        self.metric_rec = torchmetrics.Recall(num_classes=2)
        self.metric_pre = torchmetrics.Precision(num_classes=2)

        self.loss_func = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None) :
        output = self.electra(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return output

    def training_step(self, batch, batch_idx) :
        '''
        ##########################################################
        electra forward input shape information
        * input_ids.shape (batch_size, max_length)
        * attention_mask.shape (batch_size, max_length)
        * label.shape (batch_size,)
        ##########################################################
        '''

        # change label shape (list -> torch.Tensor((batch_size, 1)))
        label = batch['label'].view([-1,1])

        output = self(input_ids=batch['input_ids'].to(device),
                        attention_mask=batch['attention_mask'].to(device),
                        labels=label.to(device))
        '''
        ##########################################################
        electra forward output shape information
        * loss.shape (1,)
        * logits.shape (batch_size, config.num_labels=2)
        '''
        logits = output.logits

        loss = output.loss
        # loss = self.loss_func(logits.to(device), batch['label'].to(device))

        softmax = nn.functional.softmax(logits, dim=1)
        preds = softmax.argmax(dim=1)

        self.log("train_loss", loss, prog_bar=True)
        
        return {
            'loss' : loss,
            'pred' : preds,
            'label' : batch['label']
        }

    def training_epoch_end(self, outputs, state='train') :
        y_true = []
        y_pred = []

        for i in outputs :
            y_true += i['label'].tolist()
            y_pred += i['pred'].tolist()

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred)
        rec = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)

        # Metrics for training using Numpy
        # print(f"Type of y_pred is {type(y_pred)}")

        train_label = np.array(y_pred)
        real_label = np.array(y_true)

        TP = ((real_label == 1) & (train_label == 1)).sum()
        FP = ((real_label == 1) & (train_label == 0)).sum()
        FN = ((real_label == 0) & (train_label == 1)).sum()
        acc_sum = (real_label == train_label).sum().item()

        acc_train = acc_sum / len(y_pred)
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        F1_score = TP / (TP + 0.5*(FN+FP))

        # self.log(state+'_acc', acc, on_epoch=True, prog_bar=True)
        # self.log(state+'_precision', prec, on_epoch=True, prog_bar=True)
        # self.log(state+'_recall', rec, on_epoch=True, prog_bar=True)
        # self.log(state+'_f1', f1, on_epoch=True, prog_bar=True)

        print(f'[Epoch {self.trainer.current_epoch} {state.upper()}] Acc: {acc}, Prec: {prec}, Rec: {rec}, F1: {f1}')
        #print(f'[Epoch {self.trainer.current_epoch} {state.upper()}] Acc: {acc_train}, Prec: {precision}, Rec: {recall}, F1: {F1_score}')

    def validation_step(self, batch, batch_idx) :
        '''
        ##########################################################
        electra forward input shape information
        * input_ids.shape (batch_size, max_length)
        * attention_mask.shape (batch_size, max_length)
        ##########################################################
        '''
        output = self(input_ids=batch['input_ids'].to(device),
                        attention_mask=batch['attention_mask'].to(device))
        logits = output.logits
        preds = nn.functional.softmax(logits, dim=1).argmax(dim=1)
        labels = batch['label']

        accuracy = self.metric_acc(preds, labels)
        f1 = self.metric_f1(preds, labels)
        rec = self.metric_rec(preds, labels)
        prec = self.metric_pre(preds, labels)

        # Metrics for validation using Numpy
        val_label = preds.cpu().detach().numpy()
        real_label = labels.cpu().detach().numpy()

        TP = ((real_label == 1) & (val_label == 1)).sum()
        FP = ((real_label == 1) & (val_label == 0)).sum()
        FN = ((real_label == 0) & (val_label == 1)).sum()
        acc_sum = (real_label == val_label).sum().item()

        acc_val = acc_sum / len(val_label)
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        F1_score = TP / (TP + 0.5*(FN+FP))

        # self.log('val_accuracy', accuracy, on_epoch=True, prog_bar=True)
        # self.log('val_f1', f1, on_epoch=True, prog_bar=True)
        # self.log('val_recall', rec, on_epoch=True, prog_bar=True)
        # self.log('val_precision', prec, on_epoch=True, prog_bar=True)
        
        # return {
        #     'accuracy' : accuracy,
        #     'f1' : f1,
        #     'recall' : recall,
        #     'precision' : precision
        # }

        if str(acc_val) != 'nan' and str(precision) != 'nan' and str(recall) != 'nan' and str(F1_score) != 'nan':

          self.log('val_accuracy', acc_val, on_epoch=True, prog_bar=True)
          self.log('val_f1', F1_score, on_epoch=True, prog_bar=True)
          self.log('val_recall', recall, on_epoch=True, prog_bar=True)
          self.log('val_precision', precision, on_epoch=True, prog_bar=True)

          #print(f'This is one step: val_accuracy : {acc_val}, val_f1 : {F1_score}, val_recall : {recall}, val_precision : {precision}')

          return {
              'accuracy' : acc_val,
              'f1' : F1_score,
              'recall' : recall,
              'precision' : precision
          }

    def validation_epoch_end(self, outputs) :
        # val_acc = torch.stack([i['accuracy'] for i in outputs]).mean()
        # val_f1 = torch.stack([i['f1'] for i in outputs]).mean()
        # val_rec = torch.stack([i['recall'] for i in outputs]).mean()
        # val_pre = torch.stack([i['precision'] for i in outputs]).mean()

        #print(f"This is output in validation epoch end {outputs}")
        val_acc = []
        val_f1 = []
        val_rec = []
        val_pre = []

        for item in outputs:
          val_acc.append(item['accuracy'])
          val_f1.append(item['f1'])
          val_rec.append(item['recall'])
          val_pre.append(item['precision'])

        # val_acc = outputs['accuracy'].mean()
        # val_f1 = outputs['f1'].mean()
        # val_rec = outputs['recall'].mean()
        # val_pre = outputs['precision'].mean()

        # self.log('val_f1', val_f1, on_epoch=True, prog_bar=True)
        # self.log('val_acc', val_acc, on_epoch=True, prog_bar=True)

        # print(f'val_accuracy : {val_acc}, val_f1 : {val_f1}, val_recall : {val_rec}, val_precision : {val_pre}')
        
        print(f'val_accuracy : {sum(val_acc)/len(val_acc)}, val_f1 : {sum(val_f1)/len(val_f1)}, val_recall : {sum(val_rec)/len(val_rec)}, val_precision : {sum(val_pre)/len(val_pre)}')
    
    def test_step(self, batch, batch_idx):
        output = self(input_ids=batch['input_ids'].to(device),
                        attention_mask=batch['attention_mask'].to(device))
        logits = output.logits
        preds = nn.functional.softmax(logits, dim=1).argmax(dim=1)
        labels = batch['label']

        accuracy = self.metric_acc(preds, labels)
        f1 = self.metric_f1(preds, labels)
        recall = self.metric_rec(preds, labels)
        precision = self.metric_pre(preds, labels)

        # Metrics for test using Numpy
        test_label = preds.cpu().detach().numpy()
        real_label = labels.cpu().detach().numpy()

        TP = ((real_label == 1) & (test_label == 1)).sum()
        FP = ((real_label == 1) & (test_label == 0)).sum()
        FN = ((real_label == 0) & (test_label == 1)).sum()
        acc_sum = (real_label == test_label).sum().item()

        acc_test = acc_sum / len(test_label)
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        F1_score = TP / (TP + 0.5*(FN+FP))

        # self.log('test_accuracy', accuracy, on_epoch=True, prog_bar=True)
        # self.log('test_f1', f1, on_epoch=True, prog_bar=True)
        # self.log('test_recall', recall, on_epoch=True, prog_bar=True)
        # self.log('test_precision', precision, on_epoch=True, prog_bar=True)

        # return {
        #     'accuracy' : accuracy,
        #     'f1' : f1,
        #     'recall' : recall,
        #     'precision' : precision
        # }

        if str(acc_test) != 'nan' and str(precision) != 'nan' and str(recall) != 'nan' and str(F1_score) != 'nan':

          self.log('test_accuracy', acc_test, on_epoch=True, prog_bar=True)
          self.log('test_f1', F1_score, on_epoch=True, prog_bar=True)
          self.log('test_recall', recall, on_epoch=True, prog_bar=True)
          self.log('test_precision', precision, on_epoch=True, prog_bar=True)

          #print(f'This is one step: val_accuracy : {acc_val}, val_f1 : {F1_score}, val_recall : {recall}, val_precision : {precision}')

          return {
              'accuracy' : acc_test,
              'f1' : F1_score,
              'recall' : recall,
              'precision' : precision
          }


    def test_end(self, outputs):
        # test_acc = torch.stack([i['accuracy'] for i in outputs]).mean()
        # test_f1 = torch.stack([i['f1'] for i in outputs]).mean()
        # test_rec = torch.stack([i['recall'] for i in outputs]).mean()
        # test_pre = torch.stack([i['precision'] for i in outputs]).mean()
        # self.log('val_f1', val_f1, on_epoch=True, prog_bar=True)
        # self.log('val_acc', val_acc, on_epoch=True, prog_bar=True)

        test_acc = []
        test_f1 = []
        test_rec = []
        test_pre = []

        for item in outputs:
          test_acc.append(item['accuracy'])
          test_f1.append(item['f1'])
          test_rec.append(item['recall'])
          test_pre.append(item['precision'])

        # print(f'test_accuracy : {test_acc}, test_f1 : {test_f1}, test_recall : {test_rec}, test_precision : {test_pre}')
        print(f'test_accuracy : {sum(test_acc)/len(test_acc)}, test_f1 : {sum(test_f1)/len(test_f1)}, test_recall : {sum(test_rec)/len(test_rec)}, test_precision : {sum(test_pre)/len(test_pre)}')

        

    # def test_epoch_end(self, outputs):
    #     all_preds, all_labels = [], []
    #     for output in outputs:
    #         probs = list(output['logits'].cpu().detach().numpy()) # predicted values
    #         labels = list(output['labels'].flatten().cpu().detach().numpy())
    #         all_preds.extend(probs)
    #         all_labels.extend(labels)

    #     # you can calculate R2 here or save results as file
    #     r2 = ...
    
    # def predict_step(self, test_batch):
    #   x, y = test_batch
    #   logits = self.forward(x)
    #   return {'logits': logits, 'labels':y}

    def configure_optimizers(self) :
        optimizer = torch.optim.AdamW(self.electra.parameters(), lr=self.learning_rate)
        lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
        
        return {
            'optimizer' : optimizer,
            'lr_scheduler' : lr_scheduler
        }

In [23]:
# Main to train the model

# Initialize WandB 
wandb_logger = WandbLogger(project='Electra Classification', 
                           log_model='all')
model = ElectraClassification(learning_rate=0.0001)

wandb.watch(model)

dm = ElectraClassificationDataModule(batch_size=8, train_path='/content/Electra_classification_fake_vs_real_news/dataset/train.csv', valid_path='/content/Electra_classification_fake_vs_real_news/dataset/val.csv',
                                     test_path='/content/Electra_classification_fake_vs_real_news/dataset/test.csv',
                                max_length=512, sep=',', doc_col='text', label_col='label', num_workers=1)
dm.setup()
train_dataset = dm.train_dataloader()
valid_dataset = dm.val_dataloader()

checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_accuracy',
                                                dirpath='./sample_electra_binary_nsmc_chpt',
                                                filename='ELECTRA/{epoch:02d}-{val_accuracy:.3f}',
                                                verbose=True,
                                                save_last=True,
                                                mode='max',
                                                save_top_k=-1,
                                                )

tb_logger = pl_loggers.TensorBoardLogger(os.path.join('./sample_electra_binary_nsmc_chpt', 'tb_logs'))

lr_logger = pl.callbacks.LearningRateMonitor()

trainer = pl.Trainer(
    default_root_dir='./sample_electra_binary_nsmc_chpt/checkpoints',
    logger = wandb_logger,
    callbacks = [checkpoint_callback, lr_logger],
    max_epochs=5,
    gpus=1)

trainer.fit(model, train_dataset, valid_dataset)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/wandb.py:348: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"
Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initi

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


This is output in validation epoch end [{'accuracy': 0.625, 'f1': 0.5714285714285714, 'recall': 0.6666666666666666, 'precision': 0.5}, {'accuracy': 0.375, 'f1': 0.4444444444444444, 'recall': 0.3333333333333333, 'precision': 0.6666666666666666}]
val_accuracy : 0.5, val_f1 : 0.5079365079365079, val_recall : 0.5, val_precision : 0.5833333333333333


/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (_ResultMetric). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_no_full_state`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does

...

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:123: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:124: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:125: RuntimeWarning: invalid value encountered in true_divide


This is output in validation epoch end [{'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0, 'precision': 1.0}, {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0, 'precision': 1.0}, {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0, 'precision': 1.0}, {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0, 'precision': 1.0}, {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0, 'precision': 1.0}, {'accuracy': 0.875, 'f1': 0.8571428571428571, 'recall': 0.75, 'precision': 1.0}, {'accuracy': 0.875, 'f1': 0.8888888888888888, 'recall': 1.0, 'precision': 0.8}, {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0, 'precision': 1.0}, {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0, 'precision': 1.0}, {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0, 'precision': 1.0}, {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0, 'precision': 1.0}, {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0, 'precision': 1.0}, {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0, 'precision': 1.0}, {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0, 'precision': 1.0}, {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0,

Epoch 4, global step 28230: 'val_accuracy' reached 0.98976 (best 0.98976), saving model to '/content/sample_electra_binary_nsmc_chpt/ELECTRA/epoch=04-val_accuracy=0.990.ckpt' as top 5


[Epoch 4 TRAIN] Acc: 0.9933801226559213, Prec: 0.9932695543483486, Rec: 0.9923631123919309, F1: 0.9928161264746161


In [25]:
test_dataset = dm.test_dataloader()
trainer.test(dataloaders=test_dataset)
#trainer.test(ckpt_path=None)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1381: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  + f" You can pass `.{fn}(ckpt_path='best')` to use the best model or"
Restoring states from the checkpoint path at /content/sample_electra_binary_nsmc_chpt/ELECTRA/epoch=04-val_accuracy=0.990.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /content/sample_electra_binary_nsmc_chpt/ELECTRA/epoch=04-val_accuracy=0.990.ckpt
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument`

Testing: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (_ResultMetric). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_no_full_state`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:207: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-pack

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.9910456538200378
         test_f1            0.9913358802440259
     test_precision         0.9960028609941958
       test_recall          0.9888104866441088
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy': 0.9910456538200378,
  'test_f1': 0.9913358802440259,
  'test_precision': 0.9960028609941958,
  'test_recall': 0.9888104866441088}]

In [26]:
wandb.finish()

epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
lr-AdamW,█▆▄▃▁
test_accuracy,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_loss,▁▁█▁▃▇▂▁▁▁█▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_accuracy,▇▆▇▁█
val_f1,▆▆▇▁█
val_precision,▂▁▆█▇


### Code below this line is extraneous. 

In [ ]:
electra = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator")

# Check parameters
dm = ElectraClassificationDataModule(batch_size=8, train_path='/content/Electra_classification_fake_vs_real_news/sample_dataset/train.csv', valid_path='/content/Electra_classification_fake_vs_real_news/sample_dataset/val.csv',
                                    max_length=256, sep=',', doc_col='Tweet', label_col='is_retweet', num_workers=1)

dm.setup()

t = dm.train_dataloader()

print(t)
for idx, data in enumerate(t):
    print(idx, data['input_ids'].shape, data['attention_mask'].shape, data['label'].shape)

# Concatenate the batches ?? ********* PENDING *********** HOW TO DO THIS ?? 
#idx, data = enumerate(t)

v = dm.val_dataloader()

for idx, data in enumerate(v) :
  print(idx, data['input_ids'].shape, data['attention_mask'].shape, data['label'].shape)
  # print(idx, data['input_ids'], data['attention_mask'], data['label'])

  output = electra.forward(data['input_ids'], attention_mask=data['attention_mask'], labels=data['label'].view([-1,1]))

  print("This is the loss")
  print(output.loss)
  # print(output.loss.shape)
  # print(output.logits)
  print(output.logits.shape)

  softmax = nn.functional.softmax(output.logits, dim=1)
  print('softmax', softmax)
  pred = softmax.argmax(dim=1)
  print('pred', pred)

  y_true = data['label'].tolist()
  y_pred = pred.tolist()

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f'acc : {acc}, prec : {prec}, rec : {rec}, f1 : {f1}')
